<a href="https://colab.research.google.com/github/sifatbhuiyan0909/Dhaka-Finance-Navigator/blob/main/notebooks/06_outlier_detection_and_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded= files.upload()

Saving Dhaka-Stock-Exchange-DSE-2021.csv to Dhaka-Stock-Exchange-DSE-2021.csv
Saving Dhaka-Stock-Exchange-DSE-2020.csv to Dhaka-Stock-Exchange-DSE-2020.csv
Saving Dhaka-Stock-Exchange-DSE-2019.csv to Dhaka-Stock-Exchange-DSE-2019.csv
Saving Dhaka-Stock-Exchange-DSE-2018.csv to Dhaka-Stock-Exchange-DSE-2018.csv
Saving Dhaka-Stock-Exchange-DSE-2017.csv to Dhaka-Stock-Exchange-DSE-2017.csv
Saving Dhaka-Stock-Exchange-DSE-2016.csv to Dhaka-Stock-Exchange-DSE-2016.csv
Saving Dhaka-Stock-Exchange-DSE-2015.csv to Dhaka-Stock-Exchange-DSE-2015.csv
Saving Dhaka-Stock-Exchange-DSE-2014.csv to Dhaka-Stock-Exchange-DSE-2014.csv
Saving Dhaka-Stock-Exchange-DSE-2013.csv to Dhaka-Stock-Exchange-DSE-2013.csv
Saving Dhaka-Stock-Exchange-DSE-2012.csv to Dhaka-Stock-Exchange-DSE-2012.csv
Saving Dhaka-Stock-Exchange-DSE-2011.csv to Dhaka-Stock-Exchange-DSE-2011.csv
Saving Dhaka-Stock-Exchange-DSE-2010.csv to Dhaka-Stock-Exchange-DSE-2010.csv
Saving Dhaka-Stock-Exchange-DSE-2009.csv to Dhaka-Stock-Exchange

In [2]:
import pandas as pd
import glob
import os

# --- 1. Data Acquisition and Merging (Day 3) ---
print("--- Day 3: Browsing, Merging, and Initial Load ---")
# Finds all CSV files and prepares to merge them.
all_filenames = glob.glob(os.path.join('.', '*.csv'))
print(f"Found {len(all_filenames)} CSV files.")

# Concatenate all files into one master DataFrame
master_df = pd.concat([pd.read_csv(f, header=None) for f in all_filenames], ignore_index=True)
master_df.columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df = master_df.copy()
print("Data merged successfully.")


# --- 2. Structural & Type Cleaning (Day 4) ---
print("\n--- Day 4: Setting Index and Cleaning Data Types ---")
# Convert Date to datetime format and set it as the index
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df = df.set_index('Date')

# Clean and convert the numerical columns
for col in ['Open', 'Low', 'Volume']:
    # Remove the problematic dash/hyphen using string accessor
    df[col] = df[col].astype(str).str.replace('-', '', regex=False)

    # Convert to numeric, forcing unconvertible data to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
print("Index set and numerical columns cleaned/converted.")


# --- 3. Missing Data Handling (Day 5) ---
print("\n--- Day 5: Handling Missing Values (NaNs) ---")
# Forward-fill (ffill) to replace all remaining NaNs with the previous valid observation.
cols_to_fill = ['Open', 'High', 'Low', 'Close', 'Volume']
df[cols_to_fill] = df[cols_to_fill].ffill()

print("Missing values corrected using forward-fill.")
print("Current Data Shape:", df.shape)
print("NaN Check:\n", df.isnull().sum())

--- Day 3: Browsing, Merging, and Initial Load ---
Found 23 CSV files.
Data merged successfully.

--- Day 4: Setting Index and Cleaning Data Types ---
Index set and numerical columns cleaned/converted.

--- Day 5: Handling Missing Values (NaNs) ---
Missing values corrected using forward-fill.
Current Data Shape: (1198363, 6)
NaN Check:
 Ticker    0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [3]:
# --- START OF DAY 6 CODE ---
print("\n--- Day 6: Outlier Detection and Smoothing ---")

# 1. Calculate Daily Percentage Returns
# .pct_change() finds the percentage difference between the current day's price and the previous day's price.
df['Daily_Return'] = df['Close'].pct_change()

print("Daily Returns calculated.")


--- Day 6: Outlier Detection and Smoothing ---
Daily Returns calculated.


In [4]:
# 2. Define Outlier Boundaries (3-Sigma Rule)
# Calculate the mean (mu) and standard deviation (sigma) of the returns.
mu = df['Daily_Return'].mean()
sigma = df['Daily_Return'].std()

# Define the lower and upper bounds (3 standard deviations away from the mean).
lower_bound = mu - (3 * sigma)
upper_bound = mu + (3 * sigma)

print(f"3-Sigma Range Calculated: {lower_bound:.4f} to {upper_bound:.4f}")

3-Sigma Range Calculated: -21.2908 to 21.4294


now i am finding how many mistakes were there in the million entries from the 23 files

In [5]:
# 3. Identify, Mask, and Neutralize Outliers

# Create a mask: True for any return below the lower_bound OR above the upper_bound.
outlier_mask = (df['Daily_Return'] < lower_bound) | (df['Daily_Return'] > upper_bound)

print(f"\n*** Number of outlier days found (to be corrected): {outlier_mask.sum()} ***")

# Set the outlier price/volume values in the identified rows to NaN (breaks the error).
df.loc[outlier_mask, ['Open', 'High', 'Low', 'Close', 'Volume']] = None

# Apply ffill to replace the new NaNs with the previous day's correct value (fixes the error).
df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].ffill()

# Cleanup: Drop the temporary return column
df = df.drop(columns=['Daily_Return'])


*** Number of outlier days found (to be corrected): 547 ***
